In [ ]:
import os
#from nipype import config
#config.enable_provenance()

import nipype.pipeline.engine as pe
import nipype.interfaces.io as nio

from nipype import Workflow, Node, MapNode, Function, IdentityInterface
from nipype import DataGrabber, DataSink
from nipype.interfaces.fsl import (Merge, FLAMEO, ContrastMgr, GLM,
                                   SmoothEstimate, Cluster, ImageMaths, MultipleRegressDesign,
                                   MultiImageMaths)
import nipype.interfaces.fsl as fsl
import nipype.interfaces.utility as util
from nipype.interfaces.fsl.maths import BinaryMaths

import pandas as pd

In [ ]:
group_dir = '/om/user/annepark/projects/EF4/group_test'
rsfmri_dir = '/om/user/annepark/projects/EF4/rsfmri_final/rsfmri_2mm/'
analysis_dir = '/om/user/annepark/projects/EF4/group_analysis'


In [ ]:
def run_rsfmri_glm(group_dir):
    meta_workflow = Workflow(name='rsfmri')
    meta_workflow.base_dir = work_dir
    
    def csv_to_glm():
        import pandas as pd
        ev_file = pd.read_csv('/om/user/annepark/projects/EF4/group_analysis/design/design.csv',
                          index_col=0, sep=',|\t| ', engine='python')
        regressors = ev_file.to_dict(orient='list')
            
        contrast_file = pd.read_csv('/om/user/annepark/projects/EF4/group_analysis/design/contrasts.csv',
                                index_col=0, sep=',|\t| ', engine='python')
        
        ev_list = contrast_file.columns.tolist()
        
        contrasts = []
        for contrast in contrast_file.index.tolist():
            contrast_vec = contrast_file.loc[contrast].values.tolist()
            con = [contrast, 'T', ev_list, contrast_vec]
            contrasts.append(con)

        return regressors, contrasts

    regressors, contrasts = csv_to_glm()
    
    roi_list = pd.read_csv(os.path.join(group_dir, 'rois.txt'), header=None)
    rois = [tuple(x) for x in roi_list.values]
    
    for roi in rois:
        wf = Workflow(name=roi[1])

        rsfmri_dir = '/om/user/annepark/projects/EF4/analysis/rsfmri_final/rsfmri_2mm'
        subj_df = pd.read_csv(os.path.join(group_dir, 'sub_list.txt'), header=None, index_col=0)
        sub_list = subj_df.index.tolist()
        num_subs = len(sub_list)

        infosource = Node(IdentityInterface(fields=['subject_id']),
                             name='infosource')

        infosource.iterables = [('subject_id', sub_list)]

        datagrabber = Node(DataGrabber(infields=['subject_id'],
                                         outfields=['rest_ts_smooth', 'parc_summary', 'meants']),
                         name='datagrabber')

        datagrabber.inputs.base_directory = rsfmri_dir
        datagrabber.inputs.template = '*'

        datagrabber.inputs.field_template = {'rest_ts_smooth': '%s/resting/timeseries/target/rest_01_smooth_trans_masked.nii.gz',
                                             'parc_summary': '%s/resting/parcellations/aparc/rest_01_summary.stats',
                                             'meants': '%s/resting/parcellations/aparc/rest_01_avgwf.txt'}
        datagrabber.inputs.template_args = {'rest_ts_smooth': [['subject_id']],
                                            'parc_summary': [['subject_id']],
                                            'meants': [['subject_id']]}
        datagrabber.inputs.sort_filelist = True

        wf.connect(infosource, 'subject_id', datagrabber, 'subject_id')

        def get_meants(base_dir, subject_id, roi_fs, meants, summary_file):
            import os
            import pandas as pd
            summary = pd.read_csv(summary_file, header=None, comment='#', delim_whitespace=True, index_col=0) 
            roi_idx = summary[summary[4].str.contains(roi_fs[0])].index.values
            meants = pd.read_csv(meants, delim_whitespace=True, usecols=roi_idx-1, header=None)
            roi_dir = os.path.join(base_dir, subject_id, 'resting', 'roi_betas')
            if not os.path.exists(roi_dir):
                os.makedirs(roi_dir)
            filename = os.path.join(roi_dir, 'seed_ts.txt')
            meants.to_csv(filename, header=False, index=False)
            return filename

        get_meants = pe.Node(Function(input_names=['base_dir', 'subject_id', 'roi_fs', 
                                                   'meants', 'summary_file'],
                                          output_names=['meants'],
                                          function=get_meants),
                                  name='get_meants')
        get_meants.inputs.roi_fs = roi
        get_meants.inputs.base_dir = rsfmri_dir

        wf.connect(infosource, 'subject_id', get_meants, 'subject_id')
        wf.connect(datagrabber, 'meants', get_meants, 'meants')
        wf.connect(datagrabber, 'parc_summary', get_meants, 'summary_file')

        fsl_glm = Node(fsl.GLM(out_file='MNI_beta.nii.gz', demean=True), name='fsl_glm')

        wf.connect(datagrabber, 'rest_ts_smooth', fsl_glm, 'in_file')
        wf.connect(get_meants, 'meants', fsl_glm, 'design')

        absbin = Node(fsl.ImageMaths(op_string= '-abs -bin', out_file='absbin.nii.gz'), 
                      name='absbin')

        wf.connect(fsl_glm, 'out_file', absbin, 'in_file')

        base = '/om/user/annepark/projects/EF4/group_analysis/base.nii.gz'

        mask = pe.JoinNode(interface=fsl.MultiImageMaths(in_file=base),
                    joinsource='infosource', joinfield='operand_files', name='mask')

        mask.inputs.op_string = '-add %s ' * (num_subs)

        wf.connect(absbin, 'out_file', mask, 'operand_files')

        merge = pe.JoinNode(interface=fsl.Merge(merged_file='merge.nii.gz', dimension='t'), 
                     joinsource='infosource', joinfield='in_files', name='merge')

        wf.connect(fsl_glm, 'out_file', merge, 'in_files')
        
        thresh = Node(fsl.ImageMaths(op_string='-thr %s -bin' % (num_subs + 1)), name='thresh')
        
        wf.connect(mask, 'out_file', thresh, 'in_file')
        
        model = Node(fsl.MultipleRegressDesign(), name='model')
        model.inputs.contrasts = contrasts
        model.inputs.regressors = regressors
        
        flameo = Node(fsl.FLAMEO(run_mode='flame1'), name='flameo')
        
        wf.connect(merge, 'merged_file', flameo, 'cope_file')
        wf.connect([(model, flameo, [('design_grp', 'cov_split_file'),
                                     ('design_mat', 'design_file'),
                                     ('design_con', 't_con_file')])
                   ])
        wf.connect(thresh, 'out_file', flameo, 'mask_file')
        
        smoothest = MapNode(fsl.SmoothEstimate(), iterfield=['zstat_file'], name='smoothest')
        
        wf.connect(flameo, 'zstats', smoothest, 'zstat_file')
        wf.connect(thresh, 'out_file', smoothest, 'mask_file')
        
        #datasink = pe.Node(interface=nio.DataSink(),
        #                       name="datasink")

        meta_workflow.add_nodes([wf])
        
    return meta_workflow

  


work_dir = '/om/user/annepark/projects/EF4/test_wd4'
wf = run_rsfmri_glm(group_dir) 
#wf.base_dir = '/om/user/annepark/projects/EF4/test_wd'
wf.write_graph()
wf.run(plugin='SLURM')
